In [19]:
import os
import pickle
import numpy as np
import tensorflow as tf
import madmom

from modules.labels import get_label_vector
from modules.madmom_cnn_prep import cnn_preprocessor
from datasets import Dataset
from modules.analysis_funcs import get_idx_to_fold, get_segmented_data, get_test_peaks
from analyze_detection import evaluate

%load_ext autoreload
%autoreload 2

FPS = 100
CONTEXT = 7

# Load Madmom normalization
def cnn_normalize(frames):
    inv_std = np.load("models/bock2013pret_inv_std.npy")
    mean = np.load("models/bock2013pret_mean.npy")
    frames_normalized = (frames - np.reshape(mean, (1,80,3)))*np.reshape(inv_std, (1,80,3))
    return frames_normalized

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
ds0 = Dataset("initslurtest")
ds1 = Dataset("slurtest_add_1")

audio_fnames = ds0.get_audio_paths() + ds1.get_audio_paths()
label_fnames = ds0.get_annotation_paths() + ds1.get_annotation_paths()

audios = [madmom.audio.signal.load_wave_file(filename)[0] for filename in audio_fnames]
sample_rates = [madmom.audio.signal.load_wave_file(filename)[1] for filename in audio_fnames]
onset_schedules = [np.loadtxt(label_fname, usecols=0) for label_fname in label_fnames]

/home/simonf/.venv/onsets/lib/python3.8/site-packages/madmom/audio/signal.py:457: UserWarning: Deprecated as of version 0.16. Please use madmom.io.audio.load_wave_file instead. Will be removed in version 0.18.
  warnings.warn('Deprecated as of version 0.16. Please use madmom.io.audio.'
/home/simonf/.venv/onsets/lib/python3.8/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


In [21]:
base_path = "results/cnn-training-220409/"
folds_path = base_path + "folds.pkl"

model_name = "added-sample-gen-nostandard"

with open(folds_path, "rb") as f:
    folds = pickle.load(f)

itf = get_idx_to_fold(folds)

TOL = 0.03


In [22]:
for r in range(len(itf.keys())):
    fold = itf[r]
    print("Test recording: ", os.path.basename(audio_fnames[r]))
    print("which is in test set of fold ", fold)
    print("Loading data...")
    x = get_segmented_data(audio_fnames[r])
    training_name = "finetune-dense-dropout"
    print("Loading model...")

    #Fold-independent
    model = tf.keras.models.load_model(base_path + "fold_" + str(fold) + "_" + model_name + "_model")
    print("Predicting...")
    out = model.predict(x)
    peaks = get_test_peaks(out, 1./FPS)
    print("Evaluating...")
    [CD,FN,FP,doubles,merged] = evaluate(onset_schedules[r], peaks, tol_sec=TOL)
    print("Precision: {:.1f} %".format( 100*CD/(CD+FP)))
    print("Recall: {:.1f} %".format( 100*CD/(CD+FN)))
    print("F-score: {:.1f} %".format( 200*CD/(2*CD+FP+FN)))
    #print("{}\t{}\t{}".format(CD,FN,FP))


Test recording:  slurtest01.wav
which is in test set of fold  4
Loading data...
Loading model...


2022-04-12 19:22:01.354322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 19:22:01.362649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 19:22:01.363229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 19:22:01.365207: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Predicting...


2022-04-12 19:22:02.509293: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2022-04-12 19:22:02.734548: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Evaluating...
Precision: 84.7 %
Recall: 98.4 %
F-score: 91.0 %
Test recording:  slurtest02.wav
which is in test set of fold  3
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 84.7 %
Recall: 100.0 %
F-score: 91.7 %
Test recording:  slurtest03.wav
which is in test set of fold  2
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 89.6 %
Recall: 98.4 %
F-score: 93.8 %
Test recording:  slurtest04.wav
which is in test set of fold  0
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 89.0 %
Recall: 97.3 %
F-score: 93.0 %
Test recording:  slurtest05.wav
which is in test set of fold  1
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 75.4 %
Recall: 97.0 %
F-score: 84.8 %
Test recording:  slurtest06.wav
which is in test set of fold  4
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 95.0 %
Recall: 98.6 %
F-score: 96.8 %
Test recording:  slurtest07.wav
which is in test set of fold  3
Lo

/home/simonf/.venv/onsets/lib/python3.8/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


Loading model...
Predicting...
Evaluating...
Precision: 91.2 %
Recall: 98.6 %
F-score: 94.8 %
Test recording:  slurtest01_IR1.wav
which is in test set of fold  2
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 93.8 %
Recall: 96.8 %
F-score: 95.2 %
Test recording:  63an_start_220306.wav
which is in test set of fold  2
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 73.0 %
Recall: 96.1 %
F-score: 83.0 %
Test recording:  slurtest08_FK1.wav
which is in test set of fold  1
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 94.0 %
Recall: 100.0 %
F-score: 96.9 %
Test recording:  slurtest03_IR1.wav
which is in test set of fold  0
Loading data...


/home/simonf/.venv/onsets/lib/python3.8/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


Loading model...
Predicting...
Evaluating...
Precision: 88.2 %
Recall: 93.8 %
F-score: 90.9 %
Test recording:  stormhatten_IR1.wav
which is in test set of fold  3
Loading data...
Loading model...
Predicting...
Evaluating...
Precision: 85.7 %
Recall: 97.2 %
F-score: 91.1 %


dict_values([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4])